In [13]:
import sasoptpy as so
import pandas as pd
import os
from math import sqrt
import random
import math

In [2]:
w = 30
l = 20
df = pd.read_csv('../data/xt_pre_data.csv')
df['xbin'] = pd.cut(df.start_x, w, labels=False)
df['ybin'] = pd.cut(df.start_y, l, labels=False)
df['xbin_to'] = pd.cut(df.end_x, w, labels=False)
df['ybin_to'] = pd.cut(df.end_y, l, labels=False)

In [3]:
def get_matrix(values, xbins, ybins):
    indices = [(x,y) for x in range(xbins) for y in range(ybins)]
    x_cuts = values['xbin']
    y_cuts = values['ybin']
    bins = values.groupby([x_cuts, y_cuts]).size()
    return bins.reindex(indices, fill_value=0).unstack().fillna(0)

def get_transition_matrix(values, xbins, ybins):
    indices = [(x1,y1,x2,y2) for x1 in range(xbins) for y1 in range(ybins) for x2 in range(xbins) for y2 in range(ybins)]
    x_from = values['xbin']
    y_from = values['ybin']
    x_to = values['xbin_to']
    y_to = values['ybin_to']
    valid_occurences = values[values['result_name'] == 'success'].copy()
    occurence = values.groupby([x_from, y_from, x_to, y_to]).size()
    successful = valid_occurences.groupby([x_from, y_from, x_to, y_to]).size()
    occurence_2d = occurence.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    successful_2d = successful.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    # Only successful attempts?
    return successful_2d.div(occurence_2d.sum(axis=1), axis=0)

In [4]:
shots_df = get_matrix(df[df['type_name']=='shot'], w, l)
goals_df = get_matrix(df[(df['type_name']=='shot') & (df['result_name'] == 'success')], w, l)
moves_df = get_matrix(df[df['type_name'].isin(['dribble', 'pass', 'cross'])], w, l)
total_df = moves_df + shots_df

moves = moves_df / total_df
shots = shots_df / total_df
scores = (goals_df / shots_df).fillna(0)
T = get_transition_matrix(df[df['type_name'].isin(['dribble', 'pass', 'cross'])], w, l)

In [5]:
def base_model():
    model = so.Model(name='xThreatModel', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    model.add_constraints(
        (xT[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.set_objective(0, name='zero', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps solve solu solution.txt'
    # !{command}
    os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [6]:
m1 = base_model()

NOTE: Initialized model xThreatModel.


In [7]:
xT1 = m1.get_variable('xT')
so.get_solution_table(xT1)

,xT
"(0, 0)",0.000819
"(0, 1)",0.000819
"(0, 2)",0.001375
"(0, 3)",0.000998
"(0, 4)",0.001792
...,...
"(29, 15)",0.031315
"(29, 16)",0.018084
"(29, 17)",0.020933
"(29, 18)",0.021285


In [8]:
table1 = so.get_solution_table(xT1)
table1.index = pd.MultiIndex.from_tuples(table1.index)
table1.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000819  0.001039  0.001191  0.001107  0.001895  0.001998  0.002059   
   1   0.000819  0.001406  0.001351  0.002007  0.002083  0.002289  0.002459   
   2   0.001375  0.001562  0.001806  0.001816  0.002167  0.002677  0.002827   
   3   0.000998  0.001667  0.001978  0.002279  0.002694  0.002777  0.003081   
   4   0.001792  0.001768  0.002269  0.002645  0.003077  0.003162  0.003357   
   5   0.001286  0.002317  0.002637  0.002939  0.003232  0.003571  0.003515   
   6   0.002130  0.002329  0.002966  0.003053  0.003418  0.003373  0.003622   
   7   0.002447  0.002644  0.002851  0.003236  0.003829  0.003587  0.003928   
   8   0.002368  0.002684  0.003206  0.003436  0.003963  0.003444  0.003835   
   9   0.002372  0.002748  0.003093  0.003662  0.003566  0.003840  0.003669   
   10  0.002596  0.002691  0.003134  0.003477  0.003887  0.003655  0.004238   
   11  0.002442  0.002805  0.002975  0.003444  0.003504  0.003904  0.004082   
   12  0.001758  0.002737  0.002941  0.003105  0.003503  0.003547  0.003825   
   13  0.002113  0.002236  0.002884  0.003196  0.003352  0.003616  0.003795   
   14  0.001720  0.002382  0.002668  0.003050  0.003235  0.003481  0.003819   
   15  0.001514  0.002181  0.002432  0.002967  0.003026  0.003233  0.003535   
   16  0.001832  0.001650  0.002445  0.002733  0.002791  0.003050  0.003333   
   17  0.001387  0.001521  0.001692  0.002223  0.002673  0.003051  0.003119   
   18  0.000641  0.001419  0.001685  0.001958  0.002052  0.002473  0.002880   
   19  0.000682  0.001233  0.001162  0.001679  0.001727  0.002042  0.002408   

             7         8         9   ...        20        21        22  \
xT 0   0.002561  0.003025  0.003332  ...  0.012407  0.013644  0.015147   
   1   0.003085  0.003194  0.003876  ...  0.013611  0.015018  0.017658   
   2   0.003359  0.003625  0.003981  ...  0.014031  0.016389  0.017036   
   3   0.003488  0.003923  0.004191  ...  0.015134  0.017289  0.019040   
   4   0.003531  0.004172  0.004749  ...  0.015794  0.018457  0.020647   
   5   0.003845  0.004382  0.004907  ...  0.016062  0.019712  0.021542   
   6   0.003999  0.004532  0.005019  ...  0.016979  0.021521  0.023549   
   7   0.004045  0.004541  0.005054  ...  0.016744  0.018097  0.023452   
   8   0.004169  0.004582  0.005296  ...  0.016840  0.018312  0.022572   
   9   0.004316  0.004775  0.005453  ...  0.016226  0.018184  0.022983   
   10  0.004253  0.004806  0.005099  ...  0.016841  0.020620  0.032066   
   11  0.004385  0.004646  0.005477  ...  0.015902  0.018992  0.020989   
   12  0.004155  0.004857  0.005310  ...  0.016242  0.018253  0.026199   
   13  0.004113  0.004739  0.005208  ...  0.016280  0.018907  0.022927   
   14  0.004035  0.004565  0.005056  ...  0.016951  0.019098  0.022123   
   15  0.003940  0.004272  0.004817  ...  0.015899  0.018320  0.021068   
   16  0.003761  0.004073  0.004475  ...  0.015468  0.018062  0.019250   
   17  0.003677  0.003938  0.004225  ...  0.014180  0.016378  0.018143   
   18  0.003147  0.003653  0.004134  ...  0.013768  0.015409  0.016907   
   19  0.002842  0.002998  0.003412  ...  0.012367  0.014160  0.014828   

             23        24        25        26        27        28        29  
xT 0   0.017097  0.016113  0.016842  0.014624  0.018601  0.014720  0.011026  
   1   0.019523  0.019662  0.019227  0.019362  0.020263  0.022688  0.028479  
   2   0.021741  0.025472  0.024271  0.022737  0.021068  0.029113  0.021336  
   3   0.023102  0.022236  0.023473  0.030867  0.031010  0.029213  0.024562  
   4   0.027146  0.029748  0.031391  0.036532  0.036802  0.028654  0.029619  
   5   0.025085  0.030381  0.027323  0.034803  0.039347  0.037895  0.027091  
   6   0.025738  0.038202  0.051228  0.050339  0.081309  0.058572  0.037129  
   7   0.030438  0.027153  0.057115  0.092116  0.117294  0.097080  0.123714  
   8   0.044882  0.071577  0.075628  0.112591  0.180477  0.252921  0.132146  
   9   0.040452  0

In [9]:
def symmetric_model():
    model = so.Model(name='xThreatModel_sym', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps solve solu solution.txt'
    !{command}
    # os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [10]:
m2 = symmetric_model()

NOTE: Initialized model xThreatModel_sym.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps solve solu solution.txt (default strategy 1)
At line 1 NAME    xThreatModel_sym
At line 2 ROWS
At line 2405 COLUMNS
At line 40971 RHS
At line 41006 RANGES
At line 41007 BOUNDS
At line 42208 ENDATA
Problem xThreatModel_sym has 2401 rows, 1800 columns and 75636 elements
Coin0008I xThreatModel_sym read with 0 errors
Presolve 1800 (-601) rows, 1499 (-301) columns and 59605 (-16031) elements
Perturbing problem by 0.001% of 41.73225 - largest nonzero change 0.0002789465 ( 0.0006684195%) - largest zero change 0
Optimal - objective value 0.97127593
After Postsolve, objective 0.97127593, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.9712759291 - 2346 iterations time 0.652, Presolve 0.01
Total time (CPU seconds):       0.73   (Wallclock seconds):       0.73



In [11]:
xT2 = m2.get_variable('xT')
e2 = m2.get_variable('error')
so.get_solution_table(xT2)

,xT
"(0, 0)",0.000849
"(0, 1)",0.000855
"(0, 2)",0.001454
"(0, 3)",0.001844
"(0, 4)",0.001811
...,...
"(29, 15)",0.031187
"(29, 16)",0.024445
"(29, 17)",0.022302
"(29, 18)",0.029201


In [12]:
table2 = so.get_solution_table(xT2)
table2.index = pd.MultiIndex.from_tuples(table2.index)
table2.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000849  0.001219  0.001170  0.001692  0.001996  0.002099  0.002387   
   1   0.000855  0.001466  0.001698  0.002104  0.002216  0.002448  0.002865   
   2   0.001454  0.001639  0.001910  0.002214  0.002659  0.002804  0.003079   
   3   0.001844  0.001768  0.002448  0.002735  0.002793  0.003038  0.003286   
   4   0.001811  0.002181  0.002469  0.002973  0.003198  0.003226  0.003445   
   5   0.001422  0.002431  0.002710  0.003069  0.003327  0.003667  0.003585   
   6   0.002210  0.002276  0.003068  0.003260  0.003508  0.003615  0.003683   
   7   0.002531  0.002790  0.002998  0.003301  0.003888  0.003655  0.003984   
   8   0.002486  0.002859  0.003287  0.003501  0.003999  0.003492  0.004075   
   9   0.002657  0.002763  0.003203  0.003730  0.003919  0.003884  0.004272   
   10  0.002657  0.002763  0.003203  0.003730  0.003919  0.003884  0.004272   
   11  0.002486  0.002859  0.003287  0.003501  0.003999  0.003492  0.004075   
   12  0.002531  0.002790  0.002998  0.003301  0.003888  0.003655  0.003984   
   13  0.002210  0.002276  0.003068  0.003260  0.003508  0.003615  0.003683   
   14  0.001422  0.002431  0.002710  0.003069  0.003327  0.003667  0.003585   
   15  0.001811  0.002181  0.002469  0.002973  0.003198  0.003226  0.003445   
   16  0.001844  0.001768  0.002448  0.002735  0.002793  0.003038  0.003286   
   17  0.001454  0.001639  0.001910  0.002214  0.002659  0.002804  0.003079   
   18  0.000855  0.001466  0.001698  0.002104  0.002216  0.002448  0.002865   
   19  0.000849  0.001219  0.001170  0.001692  0.001996  0.002099  0.002387   

             7         8         9   ...        20        21        22  \
xT 0   0.002789  0.003128  0.003337  ...  0.012456  0.013894  0.015210   
   1   0.003191  0.003569  0.003957  ...  0.013952  0.015313  0.017351   
   2   0.003576  0.003707  0.004046  ...  0.014272  0.016797  0.018717   
   3   0.003576  0.003986  0.004372  ...  0.015381  0.018564  0.019303   
   4   0.003610  0.004231  0.004805  ...  0.016023  0.018686  0.021056   
   5   0.003912  0.004502  0.004949  ...  0.016239  0.019342  0.022513   
   6   0.004074  0.004679  0.005073  ...  0.016500  0.019269  0.023377   
   7   0.004120  0.004587  0.005254  ...  0.016969  0.018181  0.023481   
   8   0.004203  0.004626  0.005319  ...  0.016008  0.018531  0.022667   
   9   0.004278  0.004800  0.005472  ...  0.016418  0.020744  0.023211   
   10  0.004278  0.004800  0.005472  ...  0.016418  0.020744  0.023211   
   11  0.004203  0.004626  0.005319  ...  0.016008  0.018531  0.022667   
   12  0.004120  0.004587  0.005254  ...  0.016969  0.018181  0.023481   
   13  0.004074  0.004679  0.005073  ...  0.016500  0.019269  0.023377   
   14  0.003912  0.004502  0.004949  ...  0.016239  0.019342  0.022513   
   15  0.003610  0.004231  0.004805  ...  0.016023  0.018686  0.021056   
   16  0.003576  0.003986  0.004372  ...  0.015381  0.018564  0.019303   
   17  0.003576  0.003707  0.004046  ...  0.014272  0.016797  0.018717   
   18  0.003191  0.003569  0.003957  ...  0.013952  0.015313  0.017351   
   19  0.002789  0.003128  0.003337  ...  0.012456  0.013894  0.015210   

             23        24        25        26        27        28        29  
xT 0   0.017092  0.016837  0.019009  0.017702  0.020104  0.024683  0.024610  
   1   0.019741  0.021879  0.020873  0.023188  0.021124  0.023256  0.029201  
   2   0.021828  0.022106  0.024644  0.025046  0.019211  0.029012  0.022302  
   3   0.022960  0.023909  0.025616  0.030560  0.030708  0.032650  0.024445  
   4   0.023273  0.030064  0.032691  0.032596  0.036711  0.028399  0.031187  
   5   0.025676  0.030951  0.028634  0.045706  0.043800  0.037811  0.027320  
   6   0.026204  0.036131  0.051253  0.052083  0.104619  0.055630  0.039035  
   7   0.032443  0.041530  0.067756  0.081718  0.141742  0.095221  0.037048  
   8   0.041270  0.072168  0.076317  0.113288  0.200938  0.238083  0.332646  
   9   0.041138  0

In [14]:
# def dist(c1, c2):
#     return sqrt((c1-(w-1))**2 + (c2-(l-1)/2)**2)

def get_distance(c1,c2):
    # get mid point of the grid
    rx = 105/w*c1 + 105/w/2
    ry = 68/l*c2 + 68/l/2
    # rx = 105 * x # actual x
    # ry = 68 * y # actual y
    gc = {'x': 105, 'y': 34} # goal center
    distance = math.sqrt(math.pow(gc['x']-rx,2) + math.pow(gc['y']-ry,2))
    return distance

def get_angle(c1,c2):
    # get mid point of the grid
    rx = 105/w*c1 + 105/w/2
    ry = 68/l*c2 + 68/l/2
    # rx = 105 * x # actual x
    # ry = 68 * y # actual y
    post1 = {'x': 105, 'y': (68-7.32)/2}
    post2 = {'x': 105, 'y': 68-(68-7.32)/2}
    angle = abs(math.degrees(math.atan2(post1['y']-ry, post1['x']-rx) - math.atan2(post2['y']-ry, post2['x']-rx)))
    return angle

def sym_incremental_model():
    model = so.Model(name='xThreatModel_sym_inc', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if get_distance(x,y) < get_distance(z,w) and get_angle(x,y) > get_angle(z,w)), name='better_grid')
    # model.add_constraints(
    #     (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and x==z), name='same_row')
    # model.add_constraints(
    #     (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and y==w), name='same_col')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps presolve off solve solu solution.txt'
    !{command}
    #os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [15]:
m3 = sym_incremental_model()

NOTE: Initialized model xThreatModel_sym_inc.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps presolve off solve solu solution.txt (default strategy 1)
At line 1 NAME    xThreatModel_sym_inc
At line 2 ROWS
At line 156466 COLUMNS
At line 349092 RHS
At line 349127 RANGES
At line 349128 BOUNDS
At line 350329 ENDATA
Problem xThreatModel_sym_inc has 156462 rows, 1800 columns and 383758 elements
Coin0008I xThreatModel_sym_inc read with 0 errors
Option for presolve changed from on to off
would have 1200 free columns in primal, 1201 in dual
Dual of model has 1800 rows and 156462 columns
Clp0014I Perturbing problem by 0.001% of 31.414043 - largest nonzero change 0 ( 0%) - largest zero change 6.5706642e-05
Clp0014I Perturbing problem by 0.001% of 0.35084148 - largest nonzero change 0 ( 0%) - largest zero change 2.9992124e-05
Clp0006I 1259  Obj -0.29594717 Dual inf 1.3863289e+08 (87553)
Clp0006I 1684  Obj -0.8179587 Dual inf 1.4877725e+08 (6

In [17]:
xT3 = m3.get_variable('xT')
e3 = m3.get_variable('error')
so.get_solution_table(xT3)
for y in range(l):
    print(y, xT3[11,y].get_value())

0 0.004047253
1 0.0047400624
2 0.0050910831
3 0.0054485573
4 0.0054485573
5 0.0058005202
6 0.0060586566
7 0.006276175
8 0.0064718953
9 0.0065041431
10 0.0065041431
11 0.0064718953
12 0.006276175
13 0.0060586566
14 0.0058005202
15 0.0054485573
16 0.0054485573
17 0.0050910831
18 0.0047400624
19 0.004047253


In [18]:
with pd.option_context('display.max_rows', None):
    print(so.get_solution_table(xT1, xT2, xT3))

                xT        xT        xT
(0, 0)    0.000819  0.000849  0.000865
(0, 1)    0.000819  0.000855  0.000875
(0, 2)    0.001375  0.001454  0.001421
(0, 3)    0.000998  0.001844  0.001421
(0, 4)    0.001792  0.001811  0.001640
(0, 5)    0.001286  0.001422  0.001659
(0, 6)    0.002130  0.002210  0.002021
(0, 7)    0.002447  0.002531  0.002167
(0, 8)    0.002368  0.002486  0.002167
(0, 9)    0.002372  0.002657  0.002167
(0, 10)   0.002596  0.002657  0.002167
(0, 11)   0.002442  0.002486  0.002167
(0, 12)   0.001758  0.002531  0.002167
(0, 13)   0.002113  0.002210  0.002021
(0, 14)   0.001720  0.001422  0.001659
(0, 15)   0.001514  0.001811  0.001640
(0, 16)   0.001832  0.001844  0.001421
(0, 17)   0.001387  0.001454  0.001421
(0, 18)   0.000641  0.000855  0.000875
(0, 19)   0.000682  0.000849  0.000865
(1, 0)    0.001039  0.001219  0.001219
(1, 1)    0.001406  0.001466  0.001421
(1, 2)    0.001562  0.001639  0.001640
(1, 3)    0.001667  0.001768  0.001760
(1, 4)    0.001768  0.002

In [19]:
table1 = so.get_solution_table(xT1)
table1.index = pd.MultiIndex.from_tuples(table1.index)
table1.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000819  0.001039  0.001191  0.001107  0.001895  0.001998  0.002059   
   1   0.000819  0.001406  0.001351  0.002007  0.002083  0.002289  0.002459   
   2   0.001375  0.001562  0.001806  0.001816  0.002167  0.002677  0.002827   
   3   0.000998  0.001667  0.001978  0.002279  0.002694  0.002777  0.003081   
   4   0.001792  0.001768  0.002269  0.002645  0.003077  0.003162  0.003357   
   5   0.001286  0.002317  0.002637  0.002939  0.003232  0.003571  0.003515   
   6   0.002130  0.002329  0.002966  0.003053  0.003418  0.003373  0.003622   
   7   0.002447  0.002644  0.002851  0.003236  0.003829  0.003587  0.003928   
   8   0.002368  0.002684  0.003206  0.003436  0.003963  0.003444  0.003835   
   9   0.002372  0.002748  0.003093  0.003662  0.003566  0.003840  0.003669   
   10  0.002596  0.002691  0.003134  0.003477  0.003887  0.003655  0.004238   
   11  0.002442  0.002805  0.002975  0.003444  0.003504  0.003904  0.004082   
   12  0.001758  0.002737  0.002941  0.003105  0.003503  0.003547  0.003825   
   13  0.002113  0.002236  0.002884  0.003196  0.003352  0.003616  0.003795   
   14  0.001720  0.002382  0.002668  0.003050  0.003235  0.003481  0.003819   
   15  0.001514  0.002181  0.002432  0.002967  0.003026  0.003233  0.003535   
   16  0.001832  0.001650  0.002445  0.002733  0.002791  0.003050  0.003333   
   17  0.001387  0.001521  0.001692  0.002223  0.002673  0.003051  0.003119   
   18  0.000641  0.001419  0.001685  0.001958  0.002052  0.002473  0.002880   
   19  0.000682  0.001233  0.001162  0.001679  0.001727  0.002042  0.002408   

             7         8         9   ...        20        21        22  \
xT 0   0.002561  0.003025  0.003332  ...  0.012407  0.013644  0.015147   
   1   0.003085  0.003194  0.003876  ...  0.013611  0.015018  0.017658   
   2   0.003359  0.003625  0.003981  ...  0.014031  0.016389  0.017036   
   3   0.003488  0.003923  0.004191  ...  0.015134  0.017289  0.019040   
   4   0.003531  0.004172  0.004749  ...  0.015794  0.018457  0.020647   
   5   0.003845  0.004382  0.004907  ...  0.016062  0.019712  0.021542   
   6   0.003999  0.004532  0.005019  ...  0.016979  0.021521  0.023549   
   7   0.004045  0.004541  0.005054  ...  0.016744  0.018097  0.023452   
   8   0.004169  0.004582  0.005296  ...  0.016840  0.018312  0.022572   
   9   0.004316  0.004775  0.005453  ...  0.016226  0.018184  0.022983   
   10  0.004253  0.004806  0.005099  ...  0.016841  0.020620  0.032066   
   11  0.004385  0.004646  0.005477  ...  0.015902  0.018992  0.020989   
   12  0.004155  0.004857  0.005310  ...  0.016242  0.018253  0.026199   
   13  0.004113  0.004739  0.005208  ...  0.016280  0.018907  0.022927   
   14  0.004035  0.004565  0.005056  ...  0.016951  0.019098  0.022123   
   15  0.003940  0.004272  0.004817  ...  0.015899  0.018320  0.021068   
   16  0.003761  0.004073  0.004475  ...  0.015468  0.018062  0.019250   
   17  0.003677  0.003938  0.004225  ...  0.014180  0.016378  0.018143   
   18  0.003147  0.003653  0.004134  ...  0.013768  0.015409  0.016907   
   19  0.002842  0.002998  0.003412  ...  0.012367  0.014160  0.014828   

             23        24        25        26        27        28        29  
xT 0   0.017097  0.016113  0.016842  0.014624  0.018601  0.014720  0.011026  
   1   0.019523  0.019662  0.019227  0.019362  0.020263  0.022688  0.028479  
   2   0.021741  0.025472  0.024271  0.022737  0.021068  0.029113  0.021336  
   3   0.023102  0.022236  0.023473  0.030867  0.031010  0.029213  0.024562  
   4   0.027146  0.029748  0.031391  0.036532  0.036802  0.028654  0.029619  
   5   0.025085  0.030381  0.027323  0.034803  0.039347  0.037895  0.027091  
   6   0.025738  0.038202  0.051228  0.050339  0.081309  0.058572  0.037129  
   7   0.030438  0.027153  0.057115  0.092116  0.117294  0.097080  0.123714  
   8   0.044882  0.071577  0.075628  0.112591  0.180477  0.252921  0.132146  
   9   0.040452  0

In [20]:
table2 = so.get_solution_table(xT2)
table2.index = pd.MultiIndex.from_tuples(table2.index)
table2.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000849  0.001219  0.001170  0.001692  0.001996  0.002099  0.002387   
   1   0.000855  0.001466  0.001698  0.002104  0.002216  0.002448  0.002865   
   2   0.001454  0.001639  0.001910  0.002214  0.002659  0.002804  0.003079   
   3   0.001844  0.001768  0.002448  0.002735  0.002793  0.003038  0.003286   
   4   0.001811  0.002181  0.002469  0.002973  0.003198  0.003226  0.003445   
   5   0.001422  0.002431  0.002710  0.003069  0.003327  0.003667  0.003585   
   6   0.002210  0.002276  0.003068  0.003260  0.003508  0.003615  0.003683   
   7   0.002531  0.002790  0.002998  0.003301  0.003888  0.003655  0.003984   
   8   0.002486  0.002859  0.003287  0.003501  0.003999  0.003492  0.004075   
   9   0.002657  0.002763  0.003203  0.003730  0.003919  0.003884  0.004272   
   10  0.002657  0.002763  0.003203  0.003730  0.003919  0.003884  0.004272   
   11  0.002486  0.002859  0.003287  0.003501  0.003999  0.003492  0.004075   
   12  0.002531  0.002790  0.002998  0.003301  0.003888  0.003655  0.003984   
   13  0.002210  0.002276  0.003068  0.003260  0.003508  0.003615  0.003683   
   14  0.001422  0.002431  0.002710  0.003069  0.003327  0.003667  0.003585   
   15  0.001811  0.002181  0.002469  0.002973  0.003198  0.003226  0.003445   
   16  0.001844  0.001768  0.002448  0.002735  0.002793  0.003038  0.003286   
   17  0.001454  0.001639  0.001910  0.002214  0.002659  0.002804  0.003079   
   18  0.000855  0.001466  0.001698  0.002104  0.002216  0.002448  0.002865   
   19  0.000849  0.001219  0.001170  0.001692  0.001996  0.002099  0.002387   

             7         8         9   ...        20        21        22  \
xT 0   0.002789  0.003128  0.003337  ...  0.012456  0.013894  0.015210   
   1   0.003191  0.003569  0.003957  ...  0.013952  0.015313  0.017351   
   2   0.003576  0.003707  0.004046  ...  0.014272  0.016797  0.018717   
   3   0.003576  0.003986  0.004372  ...  0.015381  0.018564  0.019303   
   4   0.003610  0.004231  0.004805  ...  0.016023  0.018686  0.021056   
   5   0.003912  0.004502  0.004949  ...  0.016239  0.019342  0.022513   
   6   0.004074  0.004679  0.005073  ...  0.016500  0.019269  0.023377   
   7   0.004120  0.004587  0.005254  ...  0.016969  0.018181  0.023481   
   8   0.004203  0.004626  0.005319  ...  0.016008  0.018531  0.022667   
   9   0.004278  0.004800  0.005472  ...  0.016418  0.020744  0.023211   
   10  0.004278  0.004800  0.005472  ...  0.016418  0.020744  0.023211   
   11  0.004203  0.004626  0.005319  ...  0.016008  0.018531  0.022667   
   12  0.004120  0.004587  0.005254  ...  0.016969  0.018181  0.023481   
   13  0.004074  0.004679  0.005073  ...  0.016500  0.019269  0.023377   
   14  0.003912  0.004502  0.004949  ...  0.016239  0.019342  0.022513   
   15  0.003610  0.004231  0.004805  ...  0.016023  0.018686  0.021056   
   16  0.003576  0.003986  0.004372  ...  0.015381  0.018564  0.019303   
   17  0.003576  0.003707  0.004046  ...  0.014272  0.016797  0.018717   
   18  0.003191  0.003569  0.003957  ...  0.013952  0.015313  0.017351   
   19  0.002789  0.003128  0.003337  ...  0.012456  0.013894  0.015210   

             23        24        25        26        27        28        29  
xT 0   0.017092  0.016837  0.019009  0.017702  0.020104  0.024683  0.024610  
   1   0.019741  0.021879  0.020873  0.023188  0.021124  0.023256  0.029201  
   2   0.021828  0.022106  0.024644  0.025046  0.019211  0.029012  0.022302  
   3   0.022960  0.023909  0.025616  0.030560  0.030708  0.032650  0.024445  
   4   0.023273  0.030064  0.032691  0.032596  0.036711  0.028399  0.031187  
   5   0.025676  0.030951  0.028634  0.045706  0.043800  0.037811  0.027320  
   6   0.026204  0.036131  0.051253  0.052083  0.104619  0.055630  0.039035  
   7   0.032443  0.041530  0.067756  0.081718  0.141742  0.095221  0.037048  
   8   0.041270  0.072168  0.076317  0.113288  0.200938  0.238083  0.332646  
   9   0.041138  0

In [21]:
table3 = so.get_solution_table(xT3)
table3.index = pd.MultiIndex.from_tuples(table3.index)
table3.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000865  0.001219  0.001421  0.002021  0.002167  0.002495  0.002704   
   1   0.000875  0.001421  0.001697  0.002167  0.002495  0.002704  0.002974   
   2   0.001421  0.001640  0.002167  0.002495  0.002704  0.002942  0.002974   
   3   0.001421  0.001760  0.002167  0.002495  0.002704  0.002974  0.003157   
   4   0.001640  0.002167  0.002495  0.002704  0.002942  0.003134  0.003326   
   5   0.001659  0.002167  0.002495  0.002704  0.002974  0.003157  0.003484   
   6   0.002021  0.002167  0.002704  0.002942  0.002974  0.003267  0.003499   
   7   0.002167  0.002495  0.002704  0.002942  0.003134  0.003326  0.003616   
   8   0.002167  0.002495  0.002704  0.002974  0.003134  0.003326  0.003666   
   9   0.002167  0.002495  0.002704  0.002974  0.003157  0.003484  0.003666   
   10  0.002167  0.002495  0.002704  0.002974  0.003157  0.003484  0.003666   
   11  0.002167  0.002495  0.002704  0.002974  0.003134  0.003326  0.003666   
   12  0.002167  0.002495  0.002704  0.002942  0.003134  0.003326  0.003616   
   13  0.002021  0.002167  0.002704  0.002942  0.002974  0.003267  0.003499   
   14  0.001659  0.002167  0.002495  0.002704  0.002974  0.003157  0.003484   
   15  0.001640  0.002167  0.002495  0.002704  0.002942  0.003134  0.003326   
   16  0.001421  0.001760  0.002167  0.002495  0.002704  0.002974  0.003157   
   17  0.001421  0.001640  0.002167  0.002495  0.002704  0.002942  0.002974   
   18  0.000875  0.001421  0.001697  0.002167  0.002495  0.002704  0.002974   
   19  0.000865  0.001219  0.001421  0.002021  0.002167  0.002495  0.002704   

             7         8         9   ...        20        21        22  \
xT 0   0.002974  0.003157  0.003352  ...  0.011024  0.012469  0.013176   
   1   0.003134  0.003352  0.003916  ...  0.011936  0.013176  0.015364   
   2   0.003352  0.003722  0.004014  ...  0.013176  0.015364  0.016819   
   3   0.003554  0.003852  0.004243  ...  0.014476  0.016393  0.017989   
   4   0.003554  0.004087  0.004659  ...  0.015364  0.017841  0.019983   
   5   0.003744  0.004277  0.004803  ...  0.016819  0.018889  0.021197   
   6   0.003875  0.004408  0.004891  ...  0.017057  0.021197  0.023577   
   7   0.003902  0.004423  0.004938  ...  0.017989  0.021197  0.026299   
   8   0.004028  0.004436  0.005149  ...  0.018395  0.021197  0.026299   
   9   0.004075  0.004601  0.005188  ...  0.018889  0.021349  0.032823   
   10  0.004075  0.004601  0.005188  ...  0.018889  0.021349  0.032823   
   11  0.004028  0.004436  0.005149  ...  0.018395  0.021197  0.026299   
   12  0.003902  0.004423  0.004938  ...  0.017989  0.021197  0.026299   
   13  0.003875  0.004408  0.004891  ...  0.017057  0.021197  0.023577   
   14  0.003744  0.004277  0.004803  ...  0.016819  0.018889  0.021197   
   15  0.003554  0.004087  0.004659  ...  0.015364  0.017841  0.019983   
   16  0.003554  0.003852  0.004243  ...  0.014476  0.016393  0.017989   
   17  0.003352  0.003722  0.004014  ...  0.013176  0.015364  0.016819   
   18  0.003134  0.003352  0.003916  ...  0.011936  0.013176  0.015364   
   19  0.002974  0.003157  0.003352  ...  0.011024  0.012469  0.013176   

             23        24        25        26        27        28        29  
xT 0   0.015033  0.014553  0.015271  0.013185  0.016419  0.016419  0.010670  
   1   0.016819  0.017989  0.017618  0.018403  0.016419  0.018130  0.019258  
   2   0.018395  0.020161  0.021325  0.021174  0.019258  0.023048  0.019327  
   3   0.021197  0.021325  0.021738  0.023048  0.024581  0.027523  0.023454  
   4   0.022343  0.026299  0.030852  0.030921  0.027523  0.027523  0.027053  
   5   0.023953  0.030266  0.030921  0.035300  0.038212  0.037032  0.027053  
   6   0.026299  0.039122  0.051379  0.052021  0.104485  0.056014  0.038729  
   7   0.033024  0.042099  0.067939  0.081938  0.142605  0.165576  0.038729  
   8   0.042099  0.058791  0.076288  0.104485  0.201311  0.238766  0.332813  
   9   0.042168  0

In [27]:
import json

In [42]:
d = table3.to_dict()['xT']
e = {str(key): val for (key,val) in d.items()}

In [43]:
with open("xT3.json", "w") as f:
    json.dump(e, f)

In [22]:
# Future Idea
def optimal_xT(xT_vals):
    model = so.Model('optimal_flow')
    indices = [(x,y) for x in range(w) for y in range(l)]
    nodes = ['source'] + indices + ['sink']
    
    use = model.add_variables(indices, name='use', vartype=so.binary)
    flow = model.add_variables(nodes, nodes, name='flow', vartype=so.binary)

    model.add_constraint(flow['source',7,5] == 1, name='initial_node')
    inflow = {(x,y): so.expr_sum(flow[i,j,x,y] for (i,j) in indices) + flow['source',x,y] for (x,y) in indices}
    outflow = {(x,y): so.expr_sum(flow[x,y,i,j] for (i,j) in indices) + flow[x,y,'sink'] for (x,y) in indices}
    model.add_constraints((inflow[x,y] == outflow[x,y] for (x,y) in indices), name='flow_balance')
    model.add_constraints((use[x,y] == inflow[x,y] for (x,y) in indices), name='use_if_flow')
    model.add_constraint(so.expr_sum(flow[x,y,'sink'] for (x,y) in indices) == 1, name='take_shot')
    model.add_constraint(so.expr_sum(use[x,y] for (x,y) in indices) == 6, name='limit_moves')
    model.add_constraints((flow[x,y,x,y] == 0 for (x,y) in indices), name='no_self_move')

    ## Replace this with probability matrix!!
    pass_ratings = so.expr_sum(T.loc[(x,y),(z,w)] * (xt_vals.iloc[z,w] - xt_vals.iloc[x,y]) * flow[x,y,z,w] for (x,y) in indices for (z,w) in indices)
    shot_rating = so.expr_sum(shots.loc[x,y] * scores.loc[x,y] * flow[x,y,'sink'] for (x,y) in indices)
    model.set_objective(-1* (shot_rating + pass_ratings), name='total_rating', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps presolve off solve solu solution.txt'
    !{command}

xt_vals = table3.unstack()
optimal_xT(xt_vals)

NOTE: Initialized model optimal_flow.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps presolve off solve solu solution.txt (default strategy 1)
At line 1 NAME     optimal_flow
At line 2 ROWS
At line 583 COLUMNS
At line 75307 RHS
At line 75310 RANGES
At line 75311 BOUNDS
At line 113140 ENDATA
Problem optimal_flow has 579 rows, 37828 columns and 111553 elements
Coin0008I optimal_flow read with 0 errors
Option for presolve changed from on to off
Continuous objective value is -0.436492 - 0.07 seconds
Cgl0002I 192 variables fixed
Cgl0004I processed model has 385 rows, 37055 columns (37055 integer (37055 of which binary)) and 110591 elements
Coin3009W Conflict graph built in 0.306 seconds, density: 0.132%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 6 integers unsatisfied sum - 3
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -0.431708 iterations 816
Cbc0038I Solution found of -0.431708
Cbc